In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup()

In [2]:
import json
import operator

import devlib
import trappy
import bart

from bart.sched.SchedMultiAssert import SchedMultiAssert
from wlgen import RTA, Periodic

## Target connection

In [3]:
import os
os.environ['ANDROID_HOME'] = '/ext/android-sdk-linux/'

In [4]:
from env import TestEnv

my_conf = {

    # JUNO Linux
    "platform"    : "linux",
    "board"       : "juno",
    "host"        : "192.168.0.1",
    "username"    : "root",
    "password"    : "",
    "exclude_modules" : [ "hwmon" ],

#     # JUNO Android
#     "platform"    : "android",
#     "board"       : "juno",
#     "host"        : "192.168.0.1",
#     "exclude_modules" : [ "hwmon" ],
    

    # RT-App calibration values
    "rtapp-calib" : {
        '0': 363, '1': 138, '2': 139, '3': 352, '4': 353, '5': 361
    },

    # List of additional devlib modules to install 
    "modules" : ['cgroups', 'bl', 'cpufreq'],
    
    # List of additional binary tools to install
    "tools" : ['rt-app', 'trace-cmd'],
    
    # FTrace events to collect
    "ftrace" : {
         "events" : [
             "sched_switch"
         ],
         "buffsize" : 10240
    }
}

te = TestEnv(my_conf, force_new=True)
target = te.target

# Report target connection
logging.info('Connected to %s target', target.abi)
print "DONE"

04:23:46  INFO    :         Target - Using base path: /home/derkling/Code/lisa
04:23:46  INFO    :         Target - Loading custom (inline) target configuration
04:23:46  INFO    :         Target - Devlib modules to load: ['bl', 'cpufreq', 'cgroups']
04:23:46  INFO    :         Target - Connecting linux target:
04:23:46  INFO    :         Target -   username : root
04:23:46  INFO    :         Target -       host : 192.168.0.1
04:23:46  INFO    :         Target -   password : 
04:23:46  INFO    :         Target - Connection settings:
04:23:46  INFO    :         Target -    {'username': 'root', 'host': '192.168.0.1', 'password': ''}
04:23:50  INFO    :         Target - Initializing target workdir:
04:23:50  INFO    :         Target -    /root/devlib-target
04:24:16  INFO    :         Target - Topology:
04:24:16  INFO    :         Target -    [[0, 3, 4, 5], [1, 2]]
04:24:18  INFO    :       Platform - Loading default EM:
04:24:18  INFO    :       Platform -    /home/derkling/Code/lisa/lib

DONE


## List available Controllers

In [5]:
logging.info('%14s - Available controllers:', 'CGroup')
ssys = target.cgroups.list_subsystems()
for (n,h,g,e) in ssys:
    logging.info('%14s -    %10s (hierarchy id: %d) has %d cgroups',
                 'CGroup', n, h, g)

04:24:19  INFO    :         CGroup - Available controllers:
04:24:20  INFO    :         CGroup -        cpuset (hierarchy id: 1) has 4 cgroups
04:24:20  INFO    :         CGroup -           cpu (hierarchy id: 2) has 2 cgroups
04:24:20  INFO    :         CGroup -       cpuacct (hierarchy id: 3) has 1 cgroups
04:24:20  INFO    :         CGroup -     schedtune (hierarchy id: 4) has 1 cgroups
04:24:20  INFO    :         CGroup -         blkio (hierarchy id: 5) has 1 cgroups
04:24:20  INFO    :         CGroup -        memory (hierarchy id: 6) has 1 cgroups
04:24:20  INFO    :         CGroup -       devices (hierarchy id: 7) has 1 cgroups
04:24:20  INFO    :         CGroup -    perf_event (hierarchy id: 8) has 1 cgroups
04:24:20  INFO    :         CGroup -       hugetlb (hierarchy id: 9) has 1 cgroups
04:24:20  INFO    :         CGroup -          pids (hierarchy id: 10) has 1 cgroups


## Example of CPUSET controller usage

In [6]:
# Get a reference to the CPUSet controller
cpuset = target.cgroups.controller('cpuset')

In [7]:
# Get the list of current configured CGroups for that controller
cgroups = cpuset.list_all()
logging.info('Existing CGropups:')
for cg in cgroups:
    logging.info('  %s', cg)

04:24:20  INFO    : Existing CGropups:
04:24:20  INFO    :   /
04:24:20  INFO    :   /DEVLIB_ISOL
04:24:20  INFO    :   /LITTLE
04:24:20  INFO    :   /DEVLIB_SBOX


In [8]:
# Dump the configuraiton of each controller
for cgname in cgroups:
    #print cgname
    cgroup = cpuset.cgroup(cgname)
    attrs = cgroup.get()
    #print attrs
    cpus = attrs['cpus']
    logging.info('%s:%-15s cpus: %s', cpuset.kind, cgroup.name, cpus)
    

04:24:20  INFO    : cpuset:/               cpus: 0-5
04:24:21  INFO    : cpuset:/DEVLIB_ISOL    cpus: 0
04:24:22  INFO    : cpuset:/LITTLE         cpus: 0,3-5
04:24:23  INFO    : cpuset:/DEVLIB_SBOX    cpus: 1-5


In [9]:
# Create a LITTLE partition
cpuset_littles = cpuset.cgroup('/LITTLE')

In [10]:
# Check the attributes available for this control group
print "LITTLE:\n", json.dumps(cpuset_littles.get(), indent=4)

LITTLE:
{
    "cpu_exclusive": "0", 
    "memory_spread_page": "0", 
    "sched_load_balance": "1", 
    "cpus": "0,3-5", 
    "effective_mems": "0", 
    "mem_hardwall": "0", 
    "mem_exclusive": "0", 
    "memory_pressure": "0", 
    "effective_cpus": "0,3-5", 
    "mems": "0", 
    "sched_relax_domain_level": "-1", 
    "memory_migrate": "0", 
    "memory_spread_slab": "0"
}


In [11]:
# Tune CPUs and MEMs attributes
#   they must be initialize for the group to be usable
cpuset_littles.set(cpus=target.bl.littles, mems=0)
print "LITTLE:\n", json.dumps(cpuset_littles.get(), indent=4)

LITTLE:
{
    "cpu_exclusive": "0", 
    "memory_spread_page": "0", 
    "sched_load_balance": "1", 
    "cpus": "0,3-5", 
    "effective_mems": "0", 
    "mem_hardwall": "0", 
    "mem_exclusive": "0", 
    "memory_pressure": "0", 
    "effective_cpus": "0,3-5", 
    "mems": "0", 
    "sched_relax_domain_level": "-1", 
    "memory_migrate": "0", 
    "memory_spread_slab": "0"
}


In [12]:
# Define a periodic big (80%) task
task = Periodic(
    period_ms=100,
    duty_cycle_pct=80,
    duration_s=5).get()

# Create one task per each CPU in the target
tasks={}
for tid in enumerate(target.core_names):
    tasks['task{}'.format(tid[0])] = task

# Configure RTA to run all these tasks
rtapp = RTA(target, 'simple', calibration=te.calibration())
rtapp.conf(kind='profile', params=tasks, run_dir=target.working_directory);

04:24:25  INFO    :          WlGen - Setup new workload simple
04:24:25  INFO    :          RTApp - Workload duration defined by longest task
04:24:25  INFO    :          RTApp - Default policy: SCHED_OTHER
04:24:25  INFO    :          RTApp - ------------------------
04:24:25  INFO    :          RTApp - task [task0], sched: using default policy
04:24:25  INFO    :          RTApp -  | calibration CPU: 1
04:24:25  INFO    :          RTApp -  | loops count: 1
04:24:25  INFO    :          RTApp - + phase_000001: duration 5.000000 [s] (50 loops)
04:24:25  INFO    :          RTApp - |  period   100000 [us], duty_cycle  80 %
04:24:25  INFO    :          RTApp - |  run_time  80000 [us], sleep_time  20000 [us]
04:24:25  INFO    :          RTApp - ------------------------
04:24:25  INFO    :          RTApp - task [task1], sched: using default policy
04:24:25  INFO    :          RTApp -  | calibration CPU: 1
04:24:25  INFO    :          RTApp -  | loops count: 1
04:24:25  INFO    :          RTAp

In [13]:
# Test execution of all these tasks into the LITTLE cluster
trace = rtapp.run(ftrace=te.ftrace, cgroup=cpuset_littles.name, out_dir=te.res_dir)

04:24:30  INFO    :          WlGen - Workload execution START:
04:24:30  INFO    :          WlGen -    /root/devlib-target/bin/shutils cgroups_run_into /LITTLE /root/devlib-target/bin/rt-app /root/devlib-target/simple_00.json 2>&1
04:24:46  INFO    :          WlGen - Pulling trace file into [/home/derkling/Code/lisa/results/20161108_162419/simple_00.dat]...


In [14]:
# Check tasks residency on little clsuter
trappy.plotter.plot_trace(trace)

In [15]:
# Compute and visualize tasks residencies on LITTLE clusterh CPUs
s = SchedMultiAssert(trappy.FTrace(trace), te.topology, execnames=tasks.keys())
residencies = s.getResidency('cluster', target.bl.littles, percent=True)
print json.dumps(residencies, indent=4)

{
    "2258": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }, 
    "2259": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }, 
    "2260": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }, 
    "2261": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }, 
    "2262": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }, 
    "2263": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }
}


In [16]:
# Assert that ALL tasks have always executed only on LITTLE cluster
s.assertResidency('cluster', target.bl.littles,
                  99.9, operator.ge, percent=True, rank=len(residencies))

True

## Example of CPU controller usage

In [17]:
# Get a reference to the CPU controller
cpu = target.cgroups.controller('cpu')

In [18]:
# Create a big partition on that CPUS
cpu_littles = cpu.cgroup('/LITTLE')

In [19]:
# Check the attributes available for this control group
print "LITTLE:\n", json.dumps(cpu_littles.get(), indent=4)

LITTLE:
{
    "shares": "512"
}


In [20]:
# Set a 1CPU equivalent bandwidth for that CGroup
# cpu_littles.set(cfs_period_us=100000, cfs_quota_us=50000)
cpu_littles.set(shares=512)
print "LITTLE:\n", json.dumps(cpu_littles.get(), indent=4)

LITTLE:
{
    "shares": "512"
}


In [21]:
# Test execution of all these tasks into the LITTLE cluster
trace = rtapp.run(ftrace=te.ftrace, cgroup=cpu_littles.name)

04:24:57  INFO    :          WlGen - Workload execution START:
04:24:57  INFO    :          WlGen -    /root/devlib-target/bin/shutils cgroups_run_into /LITTLE /root/devlib-target/bin/rt-app /root/devlib-target/simple_00.json 2>&1
04:25:13  INFO    :          WlGen - Pulling trace file into [.//simple_00.dat]...


In [22]:
# Check tasks residency on little clsuter
trappy.plotter.plot_trace(trace)

## Example of CPUs isolation

In [23]:
# Isolate CPU0

# This works by moving all user-space tasks into a cpuset
# which does not include the specified list of CPUs to be
# isolated.
sandbox, isolated = target.cgroups.isolate(cpus=[0])

In [24]:
# Check the attributes available for the SANDBOX group
print "Sandbox:\n", json.dumps(sandbox.get(), indent=4)

Sandbox:
{
    "cpu_exclusive": "0", 
    "memory_spread_page": "0", 
    "sched_load_balance": "1", 
    "cpus": "1-5", 
    "effective_mems": "0", 
    "mem_hardwall": "0", 
    "mem_exclusive": "0", 
    "memory_pressure": "0", 
    "effective_cpus": "1-5", 
    "mems": "0", 
    "sched_relax_domain_level": "-1", 
    "memory_migrate": "0", 
    "memory_spread_slab": "0"
}


In [25]:
# Check the attributes available for the ISOLATED group
print "Isolated:\n", json.dumps(isolated.get(), indent=4)

Isolated:
{
    "cpu_exclusive": "0", 
    "memory_spread_page": "0", 
    "sched_load_balance": "1", 
    "cpus": "0", 
    "effective_mems": "0", 
    "mem_hardwall": "0", 
    "mem_exclusive": "0", 
    "memory_pressure": "0", 
    "effective_cpus": "0", 
    "mems": "0", 
    "sched_relax_domain_level": "-1", 
    "memory_migrate": "0", 
    "memory_spread_slab": "0"
}


In [26]:
# Run some workload, which is expected to not run in the ISOLATED cpus:
trace = rtapp.run(ftrace=te.ftrace)

04:25:28  INFO    :          WlGen - Workload execution START:
04:25:28  INFO    :          WlGen -    /root/devlib-target/bin/rt-app /root/devlib-target/simple_00.json 2>&1
04:25:40  INFO    :          WlGen - Pulling trace file into [.//simple_00.dat]...


In [27]:
# Check tasks was not running on ISOLATED CPUs
trappy.plotter.plot_trace(trace)

In [28]:
# Compute and visualize tasks residencies on ISOLATED CPUs
s = SchedMultiAssert(trappy.FTrace(trace), te.topology, execnames=tasks.keys())
residencies = s.getResidency('cpu', [0], percent=True)
print json.dumps(residencies, indent=4)

{
    "2389": {
        "residency": 0.0, 
        "task_name": "rt-app"
    }, 
    "2390": {
        "residency": 0.0, 
        "task_name": "rt-app"
    }, 
    "2391": {
        "residency": 0.0, 
        "task_name": "rt-app"
    }, 
    "2392": {
        "residency": 0.0, 
        "task_name": "rt-app"
    }, 
    "2393": {
        "residency": 0.0, 
        "task_name": "rt-app"
    }, 
    "2394": {
        "residency": 0.0, 
        "task_name": "rt-app"
    }
}


In [29]:
# Assert that ISOLATED CPUs was not running workload tasks
s.assertResidency('cpu', [0], 0.0, operator.eq, percent=True, rank=len(residencies))

True